## We run the methods of the paper on additional social networks
### In this notebook, we run it on large online social networks

In [1]:
# reload imported files automatically without restarting the kernel
%load_ext autoreload
%autoreload 2

In [2]:
from lib import *
from pathcensus import PathCensus
from pathcensus.nullmodels import UBCM
from pathcensus.inference import Inference
from pathcensus.utils import set_seed
from mandarina.benchmark import timer
import networkx as nx


In [3]:
@timer
def calculate_structural_measures(network_name, g, n_samples_null_model):
    """
    This function calculates the structural measures of a network and 
    its null model average values by fitting a UBCM null model to the network.
    """
    n_total = g.number_of_nodes()  # get total number of nodes
    # remove self-loops and multiedges and get largest connected component
    g = preprocess_graph(g)  ; g
    n_giant = g.number_of_nodes()  # get number of nodes in largest connected component
    degseq = sorted([d for n, d in g.degree()], reverse=True)  # get degree sequence
    dataset = ""
    network_name = network_name.split(".")[0]
    label = "large online"
    
    properties = {
            "idx": 1,
            "dataset": dataset,
            "name": network_name,
            #"graph": [g],  # get graph
            "n_nodes": n_giant,  # get number of nodes in largest connected component
            "frac_total": n_giant
            / n_total,  # get fraction of nodes in largest connected component
            "density": nx.density(g),  # get density
            "dbar": np.mean(degseq),  # get mean degree
            "dcv": np.std(degseq)
            / np.mean(degseq),  # get coefficient of variation of degree
            "dmax": np.max(degseq),  # get maximum degree
        }
    
    model = UBCM(g)  # initialize model
    model.fit()  # fit model
    model.validate()  # validate model
    # compare null model to actual graph using statistics function
    infer = Inference(g, model, statistics)
    data, null = infer.init_comparison(n=n_samples_null_model)
    original_network_values = dict(data)
    original_network_values = {'data_' + k: v for k, v in original_network_values.items()}
    null_model_mean_values = dict(null.mean(numeric_only=True)) 
    null_model_mean_values = {f'null_{n_samples_null_model}_{k}': v for k, v in null_model_mean_values.items()}
    return properties | null_model_mean_values |original_network_values | null_model_mean_values

In [4]:
OUTPUT_CSV_FILE_PATH = DATA_DIR_PATH / "structural_measures_large_online.csv"

In [5]:
file_filter = lambda file: file.is_file()
large_online_network_files = list(filter(file_filter, (DATA_DIR_PATH / "online" / "large").glob("**/*")))

results = []
for i, file in enumerate(large_online_network_files): # loop over all online large social network files
    file_size = file.stat().st_size
    print(file_size)
    if file_size > 200000000: 
        continue
    f = gml_cleaner(file) # clean gml file
    g = nx.read_gml(f, label="id") # load into networkx
    print("Running calculations for network: ", file.name) 
    # calculate measures from original network and null model
    result = calculate_structural_measures(file.name.split(".")[0] , g, n_samples_null_model=100) 
    df = pd.DataFrame(result).reset_index()
    df[df.columns[:-1]].to_csv(OUTPUT_CSV_FILE_PATH, mode='a', header=not OUTPUT_CSV_FILE_PATH.is_file()) # append result row to csv file
    results.append(result) # collect results


2533001
Running calculations for network:  NYC_restaurant_tips.gml
Function calculate_structural_measures with args (('NYC_restaurant_tips', <networkx.classes.multigraph.MultiGraph object at 0x7f3cd66e7070>), {'n_samples_null_model': 100}) took: 43.2846 seconds.
2427918
Running calculations for network:  NYC_restaurant_checkin.gml
Function calculate_structural_measures with args (('NYC_restaurant_checkin', <networkx.classes.multigraph.MultiGraph object at 0x7f3cd5024ca0>), {'n_samples_null_model': 100}) took: 27.5705 seconds.
2019618248
2113095899
649550858
1239709565
117858140
Running calculations for network:  marker_cafe.gml


In [ ]:
df = pd.concat([pd.DataFrame(result) for result in results]).reset_index() # combine all data to dataframe for analysis
df = df[df.columns[:-1]]

In [ ]:
df

,_,idx,dataset,name,n_nodes,frac_total,density,dbar,dcv,dmax,...,null_200_sim_e,null_200_comp_g,null_200_comp,null_200_comp_e,data_sim_g,data_sim,data_sim_e,data_comp_g,data_comp,data_comp_e
0,0,1,,NYC_restaurant_tips,5372,0.838066,0.000614,3.295607,1.569503,196,...,0.003327,0.004644,0.001351,0.002529,0.0,0.0,0.0,0.014416,0.003092,0.006515
1,0,1,,NYC_restaurant_checkin,4906,0.993922,0.001118,5.485936,1.044863,88,...,0.003466,0.004668,0.002053,0.003247,0.0,0.0,0.0,0.011630,0.005808,0.008769
